In [1]:
######## Particle Detection Using Tensorflow-trained Classifier #########
#
# Author: Maximilian Dreisbach
# Date: 11/05/2022
# Description: 
# This program uses a TensorFlow-trained neural network to perform particle detection.
# It loads the object detector and uses it to perform object detection on a set of images.
# It draws bounding boxes and the corresponding certainty scores around the particle images 
#in the recordings and exports the detected labels to .mat files.

## The code is an extension of the tutorial by Evan Juras [1] and the Tensorflow object detection tutorial [2]
#[1] https://github.com/EdjeElectronics/TensorFlow-Object-Detection-API-Tutorial-Train-Multiple-Objects-Windows-10
#[2] https://github.com/tensorflow/models/blob/master/research/object_detection/colab_tutorials/object_detection_tutorial.ipynb

In [2]:
# Import packages
import os
import pathlib
import cv2
import numpy as np
import tensorflow as tf
import sys
#import json
#import PIL
from PIL import Image
import xlsxwriter
import pandas as pd
import scipy.io as sio

In [3]:
# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
# Grab path to current working directory
CWD_PATH = os.getcwd()

In [4]:
# Import utilites
from utils import label_map_util
from utils import visualization_utils as vis_util

from object_detection.utils.metrics import compute_precision_recall

# Name of the directory containing the trained particle detector
MODEL_NAME = 'inference_graph'

# Path to frozen detection graph .pb file, which contains the model that is used
# for object detection.
PATH_TO_CKPT = os.path.join(CWD_PATH,MODEL_NAME,'frozen_inference_graph.pb')

# Path to label map file
PATH_TO_LABELS = os.path.join(CWD_PATH,'training','labelmap.pbtxt')

In [5]:
# Path to test images
PATH_TO_TEST_IMAGES_DIR = pathlib.Path('images/test')
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpg")))

In [6]:
# Certainty score threshold for acceptable detections (network dependent)
SCORE_THRESH = 0.8 
# Number of detection results to show
num_imageshow = 5 
# Number of classes the particle detector can identify (particle = 1, background = 0)
NUM_CLASSES = 1

In [7]:
# Load the label map.
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [8]:
# Load the Tensorflow model into memory.
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

    sess = tf.Session(graph=detection_graph)

In [9]:
# Define input and output tensors (i.e. data) for the object detection classifier

# Input tensor is the image
image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')

# Output tensors are the detection boxes, scores, and classes
# Each box represents a part of the image where a particular object was detected
detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')

# Each score represents level of confidence for each of the objects.
# The score is shown on the result image, together with the class label.
detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')

# Number of objects detected
num_detections = detection_graph.get_tensor_by_name('num_detections:0')

# name of the detection data output file
output_boxes_fname = 'boxes_output'
output_scores_fname = 'scores_output'

In [10]:
# Detection
for image_path in TEST_IMAGE_PATHS:
    with detection_graph.as_default():
        with tf.compat.v1.Session(graph=detection_graph) as sess:
            image_np = np.array(Image.open(image_path))
            image_rgb = cv2.cvtColor(image_np, cv2.COLOR_BGR2RGB)
            image_expanded = np.expand_dims(image_rgb, axis=0)

            # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
            image_np_expanded = np.expand_dims(image_np, axis=0)
            # Extract image tensor
            image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
            boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
            scores = detection_graph.get_tensor_by_name('detection_scores:0')
            classes = detection_graph.get_tensor_by_name('detection_classes:0')
            num_detections = detection_graph.get_tensor_by_name(
                'num_detections:0')
            
            # Actual detection.
            (boxes, scores, classes, num_detections) = sess.run(
                [boxes, scores, classes, num_detections],
                feed_dict={image_tensor: image_expanded})
                
            #Storing detection results
            coordinates = vis_util.return_coordinates(
                    image_rgb,
                    np.squeeze(boxes),
                    np.squeeze(classes).astype(np.int32),
                    np.squeeze(scores),
                    category_index,
                    use_normalized_coordinates=True,
                    line_thickness=8,
                    min_score_thresh=SCORE_THRESH)

            # save bounding box vector to .mat
            impath = "{}".format(image_path)
            matfile_name = "detection_results/"+output_boxes_fname+"_{}.mat".format(impath[12:-4])             
            sio.savemat(matfile_name, {'vect':coordinates})

            # save certainty scores vector to .mat
            #matfile_name = "detection_results/"+output_scores_fname+"_{}.mat".format(impath[12:-4])             
            #sio.savemat(matfile_name, {'vect':scores})

            status = "processing: {}".format(image_path)
            print(status)
                
            #Visualization of the results of a detection.    
            for i in range(num_imageshow):
                vis_util.visualize_boxes_and_labels_on_image_array(
                    image_rgb,
                    np.squeeze(boxes),
                    np.squeeze(classes).astype(np.int32),
                    np.squeeze(scores),
                    category_index,
                    use_normalized_coordinates=True,
                    line_thickness=4,
                    min_score_thresh=SCORE_THRESH)
                # Display output
                title = "object detection: {}".format(image_path)

                cv2.imshow(title, cv2.resize(image_rgb, (600, 600)))
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    cv2.destroyAllWindows()
                    break

        if cv2.waitKey(25) & 0xFF == ord('e'):
            break    
 

processing: images\test\hough_rings_00001.jpg
processing: images\test\hough_rings_00002.jpg
processing: images\test\hough_rings_00003.jpg
processing: images\test\hough_rings_00004.jpg
processing: images\test\hough_rings_00005.jpg
processing: images\test\hough_rings_00006.jpg
processing: images\test\hough_rings_00007.jpg
processing: images\test\hough_rings_00008.jpg
processing: images\test\hough_rings_00009.jpg
processing: images\test\hough_rings_00010.jpg
processing: images\test\hough_rings_00011.jpg
processing: images\test\hough_rings_00012.jpg
processing: images\test\hough_rings_00013.jpg
processing: images\test\hough_rings_00014.jpg
processing: images\test\hough_rings_00015.jpg
processing: images\test\hough_rings_00016.jpg
processing: images\test\hough_rings_00017.jpg
processing: images\test\hough_rings_00018.jpg
processing: images\test\hough_rings_00019.jpg
processing: images\test\hough_rings_00020.jpg
processing: images\test\hough_rings_00021.jpg
processing: images\test\hough_ring

KeyboardInterrupt: 